<H3> In this notebooke we actually create the VAE and train on the stock chart images

In [ ]:
import torch
import numpy as np

data_input_dir='dataset//'
image_count = 700

#load image_count images as np arrays and convert to torch tensor
dataset = torch.tensor(np.asarray([np.load(data_input_dir+'0-'+str(i)+'.npy') for i in range(image_count)]),dtype=torch.float)
print(dataset.shape)






In [ ]:
class AE(torch.nn.Module):
  def __init__(self):
    super(AE, self).__init__()
    self.encoder = torch.nn.Sequential(
        torch.nn.Conv2d(3,1,kernel_size=(5,52),stride=(5,52)),
        torch.nn.ReLU(),
        torch.nn.Conv2d(1,1,kernel_size=(5,2), stride=(5,2)),
        torch.nn.ReLU(),
        torch.nn.Conv2d(1,1,kernel_size=(4,1), stride=(4,1)),
        torch.nn.ReLU(),
      )

    self.bottleneck = torch.nn.Sequential(
        torch.nn.Linear(in_features=12, out_features=144),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=144, out_features=1*1*12),
        torch.nn.ReLU(),
      )

    self.decoder = torch.nn.Sequential(        
        torch.nn.ConvTranspose2d(1,1,kernel_size=(4,1), stride=(4,1)),
        torch.nn.ReLU(),
        torch.nn.ConvTranspose2d(1,1,kernel_size=(5,2), stride=(5,2)),
        torch.nn.ReLU(),
        torch.nn.ConvTranspose2d(1,3,kernel_size=(5,52),stride=(5,52)),
    )

  def forward(self, x):
    latent = self.encoder(x)
    latent = torch.reshape(latent, (-1,1,1,12))
    x_out = self.decoder(latent)
    return x_out

model = AE()



In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Define the loss function and optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Train the model
for epoch in range(100):
    output = model(dataset).to('cuda')
    loss = criterion(output, dataset.to('cuda'))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')


In [ ]:
output.shape

In [ ]:
np.save('test.npy',np.round(output[0,:,:,:].to('cpu').detach()))

In [ ]:
flatten = torch.nn.Flatten()
a = torch.nn.Conv2d(3,1,kernel_size=(5,52),stride=(5,52))
b = torch.nn.Conv2d(1,1,kernel_size=(5,2), stride=(5,2))
c = torch.nn.Conv2d(1,1,kernel_size=(4,1), stride=(4,1))
d = torch.nn.Linear(in_features=1*1*12, out_features=144)

out_0 = a(dataset)
out_1 = b(out_0)
out_2 = c(out_1)
flat_out_2 = flatten(out_2)
out_3 = d(flat_out_2)
print(out_0.shape, out_1.shape, out_2.shape, flat_out_2.shape, out_3.shape)

e = torch.nn.Linear(in_features=144, out_features=1*1*12)
f = torch.nn.ConvTranspose2d(1,1,kernel_size=(4,1), stride=(4,1))
g = torch.nn.ConvTranspose2d(1,1,kernel_size=(5,2), stride=(5,2))
h = torch.nn.ConvTranspose2d(1,3,kernel_size=(5,52),stride=(5,52))
i = torch.nn.ConvTranspose2d(3,3,kernel_size=(5,52),stride=(5,52))

up_0 = e(out_3)
up_0 = torch.reshape(up_0,(-1,1,1,12))
up_1 = f(up_0)
up_2 = g(up_1)
up_3 = h(up_2)
print(out_3.shape, up_0.shape, up_1.shape, up_2.shape, up_3.shape)




In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.imshow(out_0[9,0,:,:].detach())
plt.figure()
plt.imshow(out_1[9,0,:,:].detach())
plt.figure()
plt.imshow(out_2[9,0,:,:].detach())
plt.figure(figsize=(10,10))
#plt.imshow(flat_out_2[0,:].detach())
plt.imshow(out_3.detach())

In [ ]:
out_1[0,0,:,:].shape

In [ ]:
plt.figure()
plt.imshow(up_0[0,0,:,:].detach())
plt.figure()
plt.imshow(up_1[0,0,:,:].detach())
plt.figure()
plt.imshow(up_2[0,0,:,:].detach())
plt.figure()
plt.imshow(up_3[0,0,:,:].detach())